In [1]:
from pprint import pprint

import pandas as pd

In [3]:
PREFIX = '../data/top-dataset-semantic-parsing/'

with open(PREFIX + 'eval.tsv') as f:
    data = f.readlines()

In [4]:
for line in data[:10]:
    print(line)

what's to do	what 's to do	[IN:GET_EVENT what 's to do ]

What are they best place I could use to book a trip	What are they best place I could use to book a trip	[IN:UNSUPPORTED What are they best place I could use to book a trip ]

Where can we take the kids	Where can we take the kids	[IN:GET_EVENT Where can we take [SL:ATTRIBUTE_EVENT the kids ] ]

Any festivals this weekend	Any festivals this weekend	[IN:GET_EVENT Any [SL:CATEGORY_EVENT festivals ] [SL:DATE_TIME this weekend ] ]

Are there any Christmas parties this weekend	Are there any Christmas parties this weekend	[IN:GET_EVENT Are there any [SL:DATE_TIME Christmas ] [SL:CATEGORY_EVENT parties ] [SL:DATE_TIME this weekend ] ]

I need a restaurant that seems classy but is really cheap that is also downtown Manhattan but doesn't require dinner time reservations	I need a restaurant that seems classy but is really cheap that is also downtown Manhattan but doesn't require dinner time reservations	[IN:UNSUPPORTED I need a restaurant t

In [5]:
data = pd.read_table(PREFIX + 'train.tsv', names=['text', 'tokens', 'schema'])

In [6]:
data.head(10)

,text,tokens,schema
0,How long will it take to drive from Chicago to...,How long will it take to drive from Chicago to...,[IN:GET_ESTIMATED_DURATION How long will it ta...
1,Will it take shorter to get to the White House...,Will it take shorter to get to the White House...,[IN:UNSUPPORTED_NAVIGATION Will it take shorte...
2,will I make it to the beach by noon if I leave...,will I make it to the beach by noon if I leave...,[IN:GET_ESTIMATED_ARRIVAL will I make it to [S...
3,When should I leave my house to get to the Ham...,When should I leave my house to get to the Ham...,[IN:GET_ESTIMATED_DEPARTURE When should I leav...
4,I need to know if there's a lot of traffic on ...,I need to know if there 's a lot of traffic on...,[IN:GET_INFO_TRAFFIC I need to know if there '...
5,What is the shortest way home,What is the shortest way home,[IN:GET_DIRECTIONS What is the shortest way [S...
6,how long will it take me to drive to fort collins,how long will it take me to drive to fort collins,[IN:GET_ESTIMATED_DURATION how long will it ta...
7,how long would it take to ride a bus to jeddah...,how long would it take to ride a bus to jeddah...,[IN:GET_ESTIMATED_DURATION how long would it t...
8,Can you tell me what is happening at Liberty S...,Can you tell me what is happening at Liberty S...,[IN:GET_EVENT Can you tell me what is happenin...
9,traffic to Midway Airport,traffic to Midway Airport,[IN:GET_INFO_TRAFFIC traffic to [SL:DESTINATIO...


# Schema Tokenizer

In [7]:
data.schema.iloc[3]

'[IN:GET_ESTIMATED_DEPARTURE When should I leave [SL:SOURCE [IN:GET_LOCATION_HOME [SL:CONTACT my ] house ] ] to get to [SL:DESTINATION [IN:GET_LOCATION [SL:POINT_ON_MAP the Hamilton Mall ] ] ] [SL:DATE_TIME_ARRIVAL right when it opens on Saturday ] ]'

**NOTE:** We probably need to replace this tokenizer with something faster

In [42]:
def top_schema_tokenize(text):
    tokenized = ''
    for char in text:
        if char in ['[', ']']:
            char = ' ' + char + ' '
        if char in [':']:
            char = char + ' '
        tokenized += char
    tokens = tokenized.strip(' ').split(' ')
    tokens = [t for t in tokens if t != '']
    return tokens

In [9]:
data.schema.iloc[4457]

'[IN:GET_DIRECTIONS Directions from [SL:SOURCE Chicago ] to [SL:DESTINATION New York City ] . ]'

In [10]:
for i in range(4457, 4461):
    print(i, ' '.join(tokenize_schema(data.schema.iloc[i])), '\n')

4457 [ IN: GET_DIRECTIONS Directions from [ SL: SOURCE Chicago ] to [ SL: DESTINATION New York City ] . ] 

4458 [ IN: GET_ESTIMATED_DURATION how long will it take to get to [ SL: DESTINATION [ IN: GET_LOCATION [ SL: CATEGORY_LOCATION mall ] ] ] ] 

4459 [ IN: GET_INFO_TRAFFIC closed roads in [ SL: LOCATION sarasota ] ] 

4460 [ IN: GET_EVENT [ SL: CATEGORY_EVENT cooking demonstrations ] at [ SL: LOCATION [ IN: GET_LOCATION [ SL: POINT_ON_MAP Culinard ] ] ] ] 



# Schema Vocabulary

In [11]:
def top_schema_get_vocab(text):
    text = text.replace('[', '')
    text = text.replace(']', '')

    schema_tokens = {'[', ']', 'IN:', 'SL:'}

    for token in text.split(' '):
        if token[:3] in ['IN:', 'SL:']:
            schema_tokens.add(token[3:])
    return schema_tokens


In [12]:
data.schema.iloc[4457]

'[IN:GET_DIRECTIONS Directions from [SL:SOURCE Chicago ] to [SL:DESTINATION New York City ] . ]'

In [13]:
top_schema_get_vocab(data.schema.iloc[4457])

{'DESTINATION', 'GET_DIRECTIONS', 'IN:', 'SL:', 'SOURCE', '[', ']'}

In [14]:
vocab = set()

for i, line in data.iterrows():
    _vocab = top_schema_get_vocab(line.schema)
    vocab = vocab.union(_vocab)

from TOP README:
    
We collected a total of 44783 annotations with 25 intents and 36 slots, randomly split into 31279 training, 4462 validation and 9042 test utterances.


In [15]:
25 + 36 + 4

65

In [16]:
len(vocab)  # 'IN:COMBINE' and 'SL:COMBINE' collapse to a single token

64

In [17]:
vocab

{'AMOUNT',
 'ATTENDEE_EVENT',
 'ATTRIBUTE_EVENT',
 'CATEGORY_EVENT',
 'CATEGORY_LOCATION',
 'COMBINE',
 'CONTACT',
 'CONTACT_RELATED',
 'DATE_TIME',
 'DATE_TIME_ARRIVAL',
 'DATE_TIME_DEPARTURE',
 'DESTINATION',
 'GET_CONTACT',
 'GET_DIRECTIONS',
 'GET_DISTANCE',
 'GET_ESTIMATED_ARRIVAL',
 'GET_ESTIMATED_DEPARTURE',
 'GET_ESTIMATED_DURATION',
 'GET_EVENT',
 'GET_EVENT_ATTENDEE',
 'GET_EVENT_ATTENDEE_AMOUNT',
 'GET_EVENT_ORGANIZER',
 'GET_INFO_ROAD_CONDITION',
 'GET_INFO_ROUTE',
 'GET_INFO_TRAFFIC',
 'GET_LOCATION',
 'GET_LOCATION_HOME',
 'GET_LOCATION_HOMETOWN',
 'GET_LOCATION_SCHOOL',
 'GET_LOCATION_WORK',
 'GROUP',
 'IN:',
 'LOCATION',
 'LOCATION_CURRENT',
 'LOCATION_MODIFIER',
 'LOCATION_USER',
 'LOCATION_WORK',
 'METHOD_TRAVEL',
 'NAME_EVENT',
 'NEGATION',
 'OBSTRUCTION',
 'OBSTRUCTION_AVOID',
 'ORDINAL',
 'ORGANIZER_EVENT',
 'PATH',
 'PATH_AVOID',
 'POINT_ON_MAP',
 'ROAD_CONDITION',
 'ROAD_CONDITION_AVOID',
 'SEARCH_RADIUS',
 'SL:',
 'SOURCE',
 'TYPE_RELATION',
 'UNINTELLIGIBLE',
 

Full vocab here:

In [18]:
full_vocab_with_prefixes = {'IN:',
'IN:COMBINE',
 'IN:GET_CONTACT',
 'IN:GET_DIRECTIONS',
 'IN:GET_DISTANCE',
 'IN:GET_ESTIMATED_ARRIVAL',
 'IN:GET_ESTIMATED_DEPARTURE',
 'IN:GET_ESTIMATED_DURATION',
 'IN:GET_EVENT',
 'IN:GET_EVENT_ATTENDEE',
 'IN:GET_EVENT_ATTENDEE_AMOUNT',
 'IN:GET_EVENT_ORGANIZER',
 'IN:GET_INFO_ROAD_CONDITION',
 'IN:GET_INFO_ROUTE',
 'IN:GET_INFO_TRAFFIC',
 'IN:GET_LOCATION',
 'IN:GET_LOCATION_HOME',
 'IN:GET_LOCATION_HOMETOWN',
 'IN:GET_LOCATION_SCHOOL',
 'IN:GET_LOCATION_WORK',
 'IN:NEGATION',
 'IN:UNINTELLIGIBLE',
 'IN:UNSUPPORTED',
 'IN:UNSUPPORTED_EVENT',
 'IN:UNSUPPORTED_NAVIGATION',
 'IN:UPDATE_DIRECTIONS',
 'SL:',
 'SL:AMOUNT',
 'SL:ATTENDEE_EVENT',
 'SL:ATTRIBUTE_EVENT',
 'SL:CATEGORY_EVENT',
 'SL:CATEGORY_LOCATION',
 'SL:COMBINE',
 'SL:CONTACT',
 'SL:CONTACT_RELATED',
 'SL:DATE_TIME',
 'SL:DATE_TIME_ARRIVAL',
 'SL:DATE_TIME_DEPARTURE',
 'SL:DESTINATION',
 'SL:GROUP',
 'SL:LOCATION',
 'SL:LOCATION_CURRENT',
 'SL:LOCATION_MODIFIER',
 'SL:LOCATION_USER',
 'SL:LOCATION_WORK',
 'SL:METHOD_TRAVEL',
 'SL:NAME_EVENT',
 'SL:OBSTRUCTION',
 'SL:OBSTRUCTION_AVOID',
 'SL:ORDINAL',
 'SL:ORGANIZER_EVENT',
 'SL:PATH',
 'SL:PATH_AVOID',
 'SL:POINT_ON_MAP',
 'SL:ROAD_CONDITION',
 'SL:ROAD_CONDITION_AVOID',
 'SL:SEARCH_RADIUS',
 'SL:SOURCE',
 'SL:TYPE_RELATION',
 'SL:UNIT_DISTANCE',
 'SL:WAYPOINT',
 'SL:WAYPOINT_ADDED',
 'SL:WAYPOINT_AVOID',
 '[',
 ']'}

# Numericalizer

In [47]:
class SchemaTokenizer:
    def __init__(self, tokenizer_fn, vocab):
        self._vocab = vocab
        self.itos = list(vocab)
        self.stoi = {s: i for i, s in enumerate(self.itos)}

    def encode(self, schema_text, source_tokens, max_length=None, pad_to_max_length=False):
        tokens = self.tokenizer_fn(text)

        if max_length is not None:
            tokens = tokens[:max_length]
        
        delta = max_length - len(tokens)
        if pad_to_max_length and delta > 0:
            tokens += [self.pad_token] * delta

        return tokens

#     def convert_tokens_to_ids(self, tokens):
#         ids = 

In [21]:
import transformers

ENCODER_NAME = 'distilbert-base-uncased'

In [22]:
tokenizer = transformers.AutoTokenizer.from_pretrained(ENCODER_NAME, use_fast=True)

In [24]:
data.iloc[3].tokens

'When should I leave my house to get to the Hamilton Mall right when it opens on Saturday'

In [35]:
tokenizer.pad_token

'[PAD]'

In [27]:
e = tokenizer.encode(data.iloc[3].tokens)
print(e)

[101, 2043, 2323, 1045, 2681, 2026, 2160, 2000, 2131, 2000, 1996, 5226, 6670, 2157, 2043, 2009, 7480, 2006, 5095, 102]


In [ ]:
# tokenizers example: BertWordPieceTokenizer("bert-base-uncased-vocab.txt", lowercase=True)

In [34]:
data.iloc[3].schema

'[IN:GET_ESTIMATED_DEPARTURE When should I leave [SL:SOURCE [IN:GET_LOCATION_HOME [SL:CONTACT my ] house ] ] to get to [SL:DESTINATION [IN:GET_LOCATION [SL:POINT_ON_MAP the Hamilton Mall ] ] ] [SL:DATE_TIME_ARRIVAL right when it opens on Saturday ] ]'

In [65]:
_src_text = data.iloc[3].tokens
print(_src_text)
_schema = data.iloc[3].schema
print(_schema)

When should I leave my house to get to the Hamilton Mall right when it opens on Saturday
[IN:GET_ESTIMATED_DEPARTURE When should I leave [SL:SOURCE [IN:GET_LOCATION_HOME [SL:CONTACT my ] house ] ] to get to [SL:DESTINATION [IN:GET_LOCATION [SL:POINT_ON_MAP the Hamilton Mall ] ] ] [SL:DATE_TIME_ARRIVAL right when it opens on Saturday ] ]


In [68]:
src_tokens = tokenizer.encode(_src_text)
print(src_tokens)

[101, 2043, 2323, 1045, 2681, 2026, 2160, 2000, 2131, 2000, 1996, 5226, 6670, 2157, 2043, 2009, 7480, 2006, 5095, 102]


In [67]:
schema_tok = top_schema_tokenize(_schema)
print(schema_tok)

['[', 'IN:', 'GET_ESTIMATED_DEPARTURE', 'When', 'should', 'I', 'leave', '[', 'SL:', 'SOURCE', '[', 'IN:', 'GET_LOCATION_HOME', '[', 'SL:', 'CONTACT', 'my', ']', 'house', ']', ']', 'to', 'get', 'to', '[', 'SL:', 'DESTINATION', '[', 'IN:', 'GET_LOCATION', '[', 'SL:', 'POINT_ON_MAP', 'the', 'Hamilton', 'Mall', ']', ']', ']', '[', 'SL:', 'DATE_TIME_ARRIVAL', 'right', 'when', 'it', 'opens', 'on', 'Saturday', ']', ']']


In [69]:
itos = list(vocab)
stoi = {s: i for i, s in enumerate(itos)}

In [77]:
NOT_VOCAB_LEN = 1000

schema_ids = []
src_tokens_pointer = 1  # first token is [CLS]

for token in schema_tok:
    if token in vocab:
        schema_ids.append(stoi[token])
        continue
    subtokens = tokenizer.encode(token, add_special_tokens=False)
    
    for subtoken in subtokens:
        assert subtoken == src_tokens[src_tokens_pointer]
        schema_ids.append(NOT_VOCAB_LEN + src_tokens_pointer)
        src_tokens_pointer += 1

print(schema_ids)

[16, 48, 56, 1001, 1002, 1003, 1004, 16, 20, 47, 16, 48, 63, 16, 20, 32, 1005, 49, 1006, 49, 49, 1007, 1008, 1009, 16, 20, 45, 16, 48, 43, 16, 20, 8, 1010, 1011, 1012, 49, 49, 49, 16, 20, 31, 1013, 1014, 1015, 1016, 1017, 1018, 49, 49]


The final tokenizer can be found in `new_semantic_parsing.schema_tokenizer.TopSchemaTokenizer`